In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
#from ccfj import GetStationPairs
from geopy.distance import great_circle
#import shapefile
import geopandas as gp
import yaml
import math

In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import mathlib

In [3]:
from BayHunter import utils
from BayHunter import Targets
from BayHunter import MCMC_Optimizer
from BayHunter import PlotFromStorage

In [4]:
flag_project = 1 # 0--regular; 1--repartrition

In [5]:
if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']

#name_project = 'project/output_FJSJ_16-01/'               
#name_project = 'project_repartrition/repartrition_01-03/'               
#name_project = 'project_voronoi/voronoi_01-03/'         

In [6]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/


In [7]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [8]:
dir_partition = dir_project + info_basic['rdir_partition']
dir_data = dir_project + info_basic['rdir_disp_pick']

rdir_inv_McMC = 'inverion_McMC/'
dir_inv_MCMC = dir_project + rdir_inv_McMC
info_basic['rdir_inv_McMC'] = rdir_inv_McMC
if not os.path.exists(dir_inv_MCMC):
    os.makedirs(dir_inv_MCMC)

In [9]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [10]:
nums = [2]
key_subworks_all = info_basic['key_subworks']
key_subworks = []
for num in nums:
    for key_subwork in key_subworks_all:
        if str(num) == key_subwork.split('--')[0]:
            key_subworks.append(key_subwork)
            break
#key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

['2--09-03']

In [11]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = mathlib.GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 
elevation_stations_all = stainfo['elevation'].tolist()

In [12]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
num_stations = []
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    num_stations.append(len(stations_this))
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

### Demo

In [13]:
key_subwork = key_subworks[0]

In [ ]:
file_data = dir_data + 'ds_'+key_subwork+'curve.txt'
disp_data = np.loadtxt(file_data, dtype='float', unpack=True)
targets_list = []
#for mode in set(disp_data[2]):
for mode in range(0,1):
    targets_this = Targets.RayleighDispersionPhase(1./disp_data[0][disp_data[2]==mode], disp_data[1][disp_data[2]==mode])
    targets_this.moddata.plugin.set_modelparams(mode=mode+1)
    targets_list.append(targets_this)
targets = Targets.JointTarget ( targets = targets_list )

Your observed data vector exceeds the maximum of 60 periods that is allowed in SurfDisp. For forward modeling SurfDisp will reduce the samples to 60 by linear interpolation within the given period span.
From this data, the dispersion velocities to your observed periods will be determined. The precision of the data will depend on the distribution of your samples and the complexity of the input velocity-depth model.


In [ ]:
priors = {
    'vs' : (0.4,1.2),
    'layers': (1,40),
    'z': (0,0.15),
    'vpvs': (1.5,2.1),
    'rfnoise_corr':0.8,
}
initparams = {
    'nchains': 60,
    'iter_burnin': 10000,
    'iter_main': 50000,
    'savepath': dir_inv_MCMC+key_subwork+'/',
    'station': key_subwork,
}
if os.path.exists(initparams['savepath']):
    os.system('rm -rf '+initparams['savepath'])
os.makedirs(initparams['savepath'])
#initfile = 'config.ini'
#priors , initparams = utils.load_params ( initfile )

In [ ]:
utils.save_baywatch_config(targets , priors = priors , initparams = initparams)
optimizer = MCMC_Optimizer(targets , initparams = initparams , priors = priors ,random_seed = None )

In [ ]:
optimizer.mp_inversion ( nthreads =20 , baywatch = True , dtsend =0.1)

OpenBLAS blas_thread_init: pthread_create failed for thread 89 of 104: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 current, 766356 max
OpenBLAS blas_thread_init: pthread_create failed for thread 90 of 104: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 current, 766356 max
OpenBLAS blas_thread_init: pthread_create failed for thread 91 of 104: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 current, 766356 max
OpenBLAS blas_thread_init: pthread_create failed for thread 92 of 104: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 current, 766356 max
OpenBLAS blas_thread_init: pthread_create failed for thread 93 of 104: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 current, 766356 max
OpenBLAS blas_thread_init: pthread_create failed for thread 94 of 104: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 curren

In [ ]:
path = initparams ['savepath']
cfile = '% s_config.pkl' % initparams ['station']
configfile = os.path.join( path , 'data' , cfile )
obj = PlotFromStorage ( configfile )

Current data path: /shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.2_01-01/MCMC/2--09-03/data
Outlier chains from file: None


In [ ]:
obj.save_final_distribution(maxmodels = 100000, dev=0.05)

Outlier chains found with following chainindices:

[ 0.  1.  2.  3.  6.  7.  8.  9. 10. 11. 12. 13. 14. 16. 18. 19.]
> Saving posterior distribution.
/shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.2_01-01/MCMC/2--09-03/data/c_models
/shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.2_01-01/MCMC/2--09-03/data/c_likes
/shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.2_01-01/MCMC/2--09-03/data/c_misfits
/shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.2_01-01/MCMC/2--09-03/data/c_noise
/shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.2_01-01/MCMC/2--09-03/data/c_vpvs


In [ ]:
obj.save_plots ()
obj.merge_pdfs ()

/home/songsh/.local/lib/python3.9/site-packages/BayHunter/Plotting.py:612: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  models = np.array([model[~np.isnan(model)] for model in models])
/home/songsh/.local/lib/python3.9/site-packages/BayHunter/Models.py:205: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bins, vs_bin, dep_bin = np.array(data).T
/home/songsh/.local/lib/python3.9/site-packages/BayHunter/Plotting.py:482: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays wi